In [1]:
import pandas as pd
import datetime
import time
import re
# from wordsegment import load, segment
# load()
# from autocorrect import spell
import nltk
from nltk.tokenize import TweetTokenizer
# nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

### Authentification

In [2]:
import tweepy

# Your app's bearer token can be found under the Authentication Tokens section
# of the Keys and Tokens tab of your app, under the
# Twitter Developer Portal Projects & Apps page at
# https://developer.twitter.com/en/portal/projects-and-apps
bearer_token = ''

client = tweepy.Client(bearer_token=bearer_token,
                      )



### search_all_tweets

In [ ]:
def preprocessing(tweet):
    tweet =  re.sub(r' https://t.co/\w{10}', '', tweet)
    tweet =  ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)','',tweet).split())
    tweet =  ' '.join(re.sub('"!()-![]{};:+'"\,<>./?@#$%^&*_~'",'',tweet).split())
    tweet =  ' '.join(segment(tweet))
    tweet =  ' '.join([spell(w) for w in tweet.split()])

In [ ]:
text_new = []
for i in range(len(df['text'])):
               remove_link = preprocessing(df['text'][i])
               text_new.append(remove_link)
               
text_new

In [ ]:
text_new = []
for i in range(len(df['text'])):
               remove_link = re.sub(r' https://t.co/\w{10}', '', df['text'][i])
               text_new.append(remove_link)
               
text_new

In [ ]:
df['tweet_no_link'] = text_new

In [ ]:
df

In [ ]:
# https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
# https://dev.to/twitterdev/a-comprehensive-guide-for-using-the-twitter-api-v2-using-tweepy-in-python-15d9
query = 'has:geo (bounding_box:[103.8398 1.3421 103.8498 1.3521]) -is:retweet'

InitialDate= datetime.datetime(2020,1,1)
# InitialDate = '2020-01-01T00:00:00.000Z'
StartDate = (InitialDate + datetime.timedelta(days=i)).isoformat("T") + "Z"
EndDate = (InitialDate + datetime.timedelta(days=(i+5))).isoformat("T") + "Z"
tweets = client.search_all_tweets(
    query = query, 
    tweet_fields=["created_at", "lang",'geo'],
    place_fields = ['place_type', 'geo'],
    expansions=["geo.place_id"],
    start_time = StartDate,
    end_time = EndDate,
    max_results = 500
    )
    # for tweet in tweets.data:
    #     print(tweet.text)
    #     print(tweet.created_at)
    
if len(tweets.data) > 0:
    tweet_text = pd.DataFrame(data = tweets.data)
    tweets_combined = pd.concat([tweets_combined, tweet_text], ignore_index=True)
    
tweets_combined    


In [ ]:
sentences = 'Hello, world. I am terrible'
sia.polarity_scores(sentences)

In [ ]:
sentences = 'I am terrible'
sia.polarity_scores(sentences)

In [13]:
points = pd.read_csv('GE6211_pts.csv')
points

,ID,lon,lat
0,0,103.730679,1.163642
1,0,103.740679,1.163642
2,0,103.750679,1.163642
3,0,103.720679,1.173642
4,0,103.730679,1.173642
...,...,...,...
933,0,103.800679,1.473642
934,0,103.810679,1.473642
935,0,103.820679,1.473642
936,0,103.830679,1.473642


In [ ]:
points.iloc[358,:]

In [100]:
combined_tweets_6 = pd.DataFrame(columns = [])

In [ ]:
# combined_tweets = pd.read_csv('tweet_1.csv')
# combined_tweets
# 1, 96, 225, 255-259, 276-278, 330, 571-572,616,618-619,649-650,685,729-730

In [ ]:
for i in [258, 259, 276, 277, 278]:
    pt_tweets = []
    bounding_box = str(points.at[i,'lon']-0.005) + ' ' + str(points.at[i,'lat'] - 0.005) + ' ' + str(points.at[i,'lon'] + 0.005) + ' ' + str(points.at[i,'lat'] + 0.005)
    for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = '(bounding_box:[' + bounding_box + ']) -is:retweet',
#                                  user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'lang', 'text', 'public_metrics'],
                                 expansions=["geo.place_id"],
                                 start_time = '2019-01-01T00:00:00Z',
                                 end_time = '2022-01-01T00:00:00Z',
                                 max_results=500):
        time.sleep(5)
        pt_tweets.append(response)
    agg = []
    print(pt_tweets)
    
        # Loop through each response object
    for response in pt_tweets:
        if response.meta['result_count'] > 0:
             # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
            for tweet in response.data:
                    # For each tweet, find the author's information
#                 author_info = tweet.author_id
                # Put all of the information we want to keep in a single dictionary for each tweet
                agg.append({#'author_id': tweet.author_id, 
#                             'username': author_info['username'],
#                             'author_tweets': author_info['tweets'],
                            'lang': tweet.lang,
#                             'author_description': author_info['description'],
#                             'author_location': author_info['location'],
                            'text': tweet.text,
                            'created_at': tweet.created_at,
                            'retweets': tweet.public_metrics['retweet_count'],
                            'replies': tweet.public_metrics['reply_count'],
                            'likes': tweet.public_metrics['like_count'],
                            'quote_count': tweet.public_metrics['quote_count']
                           })
                    
    # Change this list of dictionaries into a dataframe
    agg = pd.DataFrame(agg)
    agg['grid'] = i
    agg['lon'] = points.at[i,'lon']
    agg['lat'] = points.at[i,'lat']
    combined_tweets_6 = pd.concat([combined_tweets_6, agg], ignore_index=True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [130]:
combined_tweets_6

,lang,text,created_at,retweets,replies,likes,quote_count,grid,lon,lat
0,en,Jog from east coast lagoon food place to garde...,2021-12-27 07:48:16+00:00,0.0,0.0,0.0,0.0,260,103.870679,1.293642
1,en,"Just posted a photo @ Bay East Gardens, Garden...",2021-12-25 15:43:06+00:00,0.0,0.0,0.0,0.0,260,103.870679,1.293642
2,ja,新型コロナで家に篭りがちなのはいかんなと思って久しぶりに海沿いを散歩。ガーデンズ・バイ・ザ・...,2021-12-18 01:20:22+00:00,0.0,1.0,4.0,0.0,260,103.870679,1.293642
3,en,New bike but my pedal already broke 😂 Waiting ...,2021-12-18 00:48:46+00:00,0.0,0.0,0.0,0.0,260,103.870679,1.293642
4,en,monday blues (skies).\n\n#punintended @ Tanjon...,2021-12-06 10:43:00+00:00,0.0,0.0,0.0,0.0,260,103.870679,1.293642
...,...,...,...,...,...,...,...,...,...,...
331632,en,I'm at Queenstown MRT Station (EW19) - @smrt_s...,2019-01-03 04:23:23+00:00,0.0,0.0,0.0,0.0,254,103.810679,1.293642
331633,en,Got him (@ Queenstown MRT Station (EW19) - @sm...,2019-01-02 04:10:27+00:00,0.0,0.0,0.0,0.0,254,103.810679,1.293642
331634,en,Walk over (@ Rainbow Centre in Singapore) http...,2019-01-02 03:48:44+00:00,0.0,0.0,0.0,0.0,254,103.810679,1.293642
331635,en,Ok (@ Queenstown MRT Station (EW19) - @smrt_si...,2019-01-02 03:48:14+00:00,0.0,0.0,0.0,0.0,254,103.810679,1.293642


In [65]:
agg

,lang,text,created_at,retweets,replies,likes,quote_count,grid,lon,lat
0,en,I'm at Selarang Park Community Supervision Cen...,2021-12-20 08:11:44+00:00,0,0,0,0,617,103.980679,1.373642
1,in,Chitty chitty Bangkok @ Lebua State Tower Hote...,2021-06-05 06:41:40+00:00,0,0,0,0,617,103.980679,1.373642
2,in,#sunset #storm #bangkok #thailand @ Lebua Stat...,2021-01-24 13:28:55+00:00,0,0,0,0,617,103.980679,1.373642
3,en,When the memories hit you ❤️ #throwbackthursda...,2020-09-23 20:19:43+00:00,0,0,0,0,617,103.980679,1.373642
4,en,Just posted a video @ Lebua State Tower Hotel ...,2020-09-20 10:09:29+00:00,0,0,0,0,617,103.980679,1.373642
5,en,Day 2 | I was nominated by @stubbornbananna to...,2020-05-24 05:52:26+00:00,0,0,0,0,617,103.980679,1.373642
6,it,"#iosonoLucca, ma in questo momento difficile m...",2020-03-15 21:40:43+00:00,0,0,0,0,617,103.980679,1.373642
7,it,"Questo è un periodo di pochi viaggi, paura, e ...",2020-03-03 20:51:08+00:00,0,0,1,0,617,103.980679,1.373642
8,in,#balconi #hotel #lebua #statetower #bangkok #t...,2020-02-02 12:48:15+00:00,0,0,0,0,617,103.980679,1.373642
9,pt,test recipe br.. https://t.co/e77VAsFv9a,2019-12-15 13:28:32+00:00,0,1,1,1,617,103.980679,1.373642


In [23]:
combined_tweets_1

,lang,text,created_at,retweets,replies,likes,quote_count,grid,lon,lat
0,und,🌊🖤 https://t.co/X5vNLurc3k,2021-09-05 05:30:50+00:00,0.0,0.0,3.0,0.0,0,103.730679,1.163642
1,en,Quantum of the sea💙 https://t.co/ySIvbONHDf,2021-08-31 02:44:15+00:00,0.0,0.0,0.0,0.0,0,103.730679,1.163642
2,en,"Sadly all good things must come to an end, as ...",2021-08-15 06:24:11+00:00,0.0,0.0,0.0,0.0,0,103.730679,1.163642
3,en,"Sadly all good things must come to an end, as ...",2021-08-15 06:22:44+00:00,0.0,0.0,2.0,0.0,0,103.730679,1.163642
4,en,South China Sea or as Vietnamese would name it...,2021-05-15 04:06:15+00:00,0.0,0.0,0.0,0.0,0,103.730679,1.163642
...,...,...,...,...,...,...,...,...,...,...
125704,en,I'm at Queenstown MRT Station (EW19) - @smrt_s...,2019-01-03 04:23:23+00:00,0.0,0.0,0.0,0.0,254,103.810679,1.293642
125705,en,Got him (@ Queenstown MRT Station (EW19) - @sm...,2019-01-02 04:10:27+00:00,0.0,0.0,0.0,0.0,254,103.810679,1.293642
125706,en,Walk over (@ Rainbow Centre in Singapore) http...,2019-01-02 03:48:44+00:00,0.0,0.0,0.0,0.0,254,103.810679,1.293642
125707,en,Ok (@ Queenstown MRT Station (EW19) - @smrt_si...,2019-01-02 03:48:14+00:00,0.0,0.0,0.0,0.0,254,103.810679,1.293642


In [24]:
# combined_tweets_1.to_csv('tweet_1.csv')

In [ ]:
combined_tweets['lang'].value_counts()

In [52]:
# combined_tweets_2.to_csv('tweet_2.csv')

In [ ]:
combined_tweets_2['lang'].value_counts()

In [69]:
# combined_tweets_3.to_csv('tweet_3.csv')

In [84]:
# combined_tweets_4.to_csv('tweet_4.csv')

In [99]:
# combined_tweets_5.to_csv('tweet_5.csv')

In [131]:
combined_tweets_6.to_csv('tweet_6.csv')